# HOMEWORK 4: Predictive Process Mining
#### Business Process Mining
Heidi Carolina Martinsaari

Github: https://github.com/h31d1/BPM_HW4

Imports:

In [1]:
import pandas as pd
import numpy as np
from preprocesses import *
import BucketFactory
import EncoderFactory
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

/Users/heidi/.local/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Download file:

In [2]:
!curl -O https://owncloud.ut.ee/owncloud/index.php/s/8DC9KXyTnJWNRgJ/download/HW4-BPIC15.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7482k  100 7482k    0     0   280k      0  0:00:26  0:00:26 --:--:--  300k  623k      0  0:00:12  0:00:03  0:00:09  625k


Import file:

In [3]:
log = pd.read_csv("HW4-BPIC15.csv")
log.head(10)

,Responsible_actor,SUMleges,Case ID,Aanleg (Uitvoeren werk of werkzaamheid),Bouw,Brandveilig gebruik (melding),Brandveilig gebruik (vergunning),Flora en Fauna,Gebiedsbescherming,Handelen in strijd met regels RO,...,question,org:resource,time:timestamp,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr
0,3442724.0,0.000,3090877,0,0,0,0,0,0,0,...,EMPTY,3148844,12/31/2009 23:00,1380,12,3,23,0.000000,0.00000,1
1,2013365.0,23847.627,5726442,0,1,0,0,0,0,0,...,EMPTY,2013365,5/12/2010 22:00,1320,5,2,22,0.000000,0.00000,1
2,560922.0,0.000,3004646,0,0,0,0,0,0,0,...,other,560922,10/3/2010 22:00,1320,10,6,22,0.000000,0.00000,1
3,560922.0,0.000,3007101,0,0,0,0,0,0,0,...,other,560749,10/3/2010 22:00,1320,10,6,22,0.000000,0.00000,1
4,560922.0,0.000,3004646,0,0,0,0,0,0,0,...,EMPTY,560922,10/5/2010 22:00,1320,10,1,22,2880.000000,2880.00000,2
5,560922.0,0.000,3007101,0,0,0,0,0,0,0,...,EMPTY,560749,10/10/2010 22:00,1320,10,6,22,10080.000000,10080.00000,2
6,2013365.0,795.792,3018942,0,1,0,0,0,0,1,...,other,560741,10/10/2010 22:00,1320,10,6,22,0.000000,0.00000,1
7,560922.0,0.000,3007101,0,0,0,0,0,0,0,...,EMPTY,560749,10/11/2010 8:22,502,10,0,8,622.033333,10702.03333,3
8,560922.0,0.000,3007101,0,0,0,0,0,0,0,...,True,560749,10/11/2010 8:22,502,10,0,8,0.216667,10702.25000,4
9,560922.0,0.000,3007101,0,0,0,0,0,0,0,...,False,560749,10/11/2010 8:22,502,10,0,8,0.600000,10702.85000,5


After familiarizing with columns, I can put down a log schema:

In [4]:
log_schema = {'case_id_col': 'Case ID',
              'timestamp_col': 'time:timestamp',
              'activity_col': 'Activity',
              'label': 'label',
              'negative_outcome': 'deviant',
              'static_num_cols': ['Responsible_actor', 'SUMleges',
                                  'Aanleg (Uitvoeren werk of werkzaamheid)', 'Bouw',
                                  'Brandveilig gebruik (melding)', 'Brandveilig gebruik (vergunning)',
                                  'Flora en Fauna', 'Gebiedsbescherming',
                                  'Handelen in strijd met regels RO', 'Inrit/Uitweg', 'Kap',
                                  'Milieu (melding)', 'Milieu (neutraal wijziging)',
                                  'Milieu (omgevingsvergunning beperkte milieutoets)',
                                  'Milieu (vergunning)', 'Monument', 'Reclame', 'Sloop'],
              'static_cat_cols': [],
              'dynamic_cat_cols': ['question'],
              'dynamic_num_cols': ['monitoringResource', 'org:resource',
                                   'timesincemidnight', 'month', 'weekday', 'hour', 
                                   'timesincelastevent','timesincecasestart', 'event_nr']
             }

In [5]:
log[log_schema.get('timestamp_col')] = pd.to_datetime(log[log_schema.get('timestamp_col')], infer_datetime_format=True)
log.head()

,Responsible_actor,SUMleges,Case ID,Aanleg (Uitvoeren werk of werkzaamheid),Bouw,Brandveilig gebruik (melding),Brandveilig gebruik (vergunning),Flora en Fauna,Gebiedsbescherming,Handelen in strijd met regels RO,...,question,org:resource,time:timestamp,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr
0,3442724.0,0.000,3090877,0,0,0,0,0,0,0,...,EMPTY,3148844,2009-12-31 23:00:00,1380,12,3,23,0.0,0.0,1
1,2013365.0,23847.627,5726442,0,1,0,0,0,0,0,...,EMPTY,2013365,2010-05-12 22:00:00,1320,5,2,22,0.0,0.0,1
2,560922.0,0.000,3004646,0,0,0,0,0,0,0,...,other,560922,2010-10-03 22:00:00,1320,10,6,22,0.0,0.0,1
3,560922.0,0.000,3007101,0,0,0,0,0,0,0,...,other,560749,2010-10-03 22:00:00,1320,10,6,22,0.0,0.0,1
4,560922.0,0.000,3004646,0,0,0,0,0,0,0,...,EMPTY,560922,2010-10-05 22:00:00,1320,10,1,22,2880.0,2880.0,2


## Task 1
You must create a new column in the log that contains a case attribute called **label** that must take a value of 1 for deviant cases or 0 for regular ones. A case is **deviant** if its duration is greater than the mean case duration of the log by 10% (i.e., mean-case-duration * 1.10). Otherwise, the case is **regular**.

In [6]:
# case level information:
cases = log.groupby(log_schema.get('case_id_col'))

In [7]:
# calculating case durations:
case_durations = cases['timesincecasestart'].max().reset_index()
case_durations.columns = ["Case ID","case_duration"]

In [8]:
# calculating mean case duration:
mean_case_dur = case_durations["case_duration"].mean()

In [9]:
# labelling cases as regular or deviant based on their duration:
case_durations["label"] = 'regular'
case_durations.loc[case_durations["case_duration"]>1.1*mean_case_dur,"label"] = 'deviant'
case_durations.sample(5)

,Case ID,case_duration,label
528,4571229,27926.566670,regular
858,5727122,36660.800000,regular
994,6216081,19521.783330,regular
1205,7459687,3533.716667,regular
756,5382350,30967.616670,regular


In [10]:
# adding case_duration and label columns to log:
log = log.set_index("Case ID").join(case_durations.set_index("Case ID")).reset_index()

## Task 2
Add a WIP column to the event log in each event. The WIP is the number of active cases that have started but not been completed.

In [11]:
# finding case_start and case_end times:
starttimes = pd.DataFrame(cases[log_schema.get('timestamp_col')].min())
starttimes.columns=["case_start"]
endtimes = pd.DataFrame(cases[log_schema.get('timestamp_col')].max()) # it is expected that last activity is instant
endtimes.columns=["case_end"]
# case level information
cases = starttimes.join(endtimes).reset_index()
cases.sample(5)

,Case ID,case_start,case_end
1021,6338218,2013-10-15 22:00:00,2013-10-31 08:25:00
333,3933353,2011-07-16 22:00:00,2011-10-21 22:00:00
852,5711808,2013-03-17 23:00:00,2013-05-07 08:44:00
712,5225326,2012-10-04 22:00:00,2012-11-18 23:00:00
649,5068949,2012-07-14 22:00:00,2012-08-12 22:00:00


In [12]:
log["wip"] = log[log_schema.get('timestamp_col')].apply(lambda x: cases[(cases.case_start <= x) & (x < cases.case_end)]["Case ID"].count())
log.sample(5)

,Case ID,Responsible_actor,SUMleges,Aanleg (Uitvoeren werk of werkzaamheid),Bouw,Brandveilig gebruik (melding),Brandveilig gebruik (vergunning),Flora en Fauna,Gebiedsbescherming,Handelen in strijd met regels RO,...,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,case_duration,label,wip
20580,4577394,560696.0,-6.744,0,1,0,0,0,0,0,...,557,3,0,9,0.050000,15017.95000,30,21600.00000,regular,62
50163,7249720,560454.0,496.527,1,1,0,0,0,0,0,...,579,6,4,9,699.466667,25179.46667,42,25179.46667,regular,44
44332,6409500,560696.0,11.802,0,0,0,0,0,0,0,...,1380,12,0,23,0.000000,69120.00000,42,69793.76667,regular,44
8975,3678295,2013365.0,386.094,0,1,0,0,0,0,0,...,540,6,2,9,660.183333,55380.18333,38,55380.41667,regular,66
15814,4186712,560696.0,0.000,0,1,0,0,0,0,0,...,532,10,4,8,0.100000,12172.48333,15,23040.00000,regular,56


### Train-test split

In [13]:
# determine min and max (truncated) prefix lengths
min_prefix_length = 1
max_prefix_length = min(40, get_pos_case_length_quantile(log, log_schema, 0.90))
min_prefix_length, max_prefix_length

(1, 40)

In [14]:
train, test = split_data_strict(log, log_schema, 0.8, split="temporal")
len(train), len(test)

(43759, 12057)

### Extract prefixes

In [15]:
dt_train_prefixes = generate_prefix_data(train, log_schema, min_prefix_length, max_prefix_length)
dt_test_prefixes = generate_prefix_data(test, log_schema, min_prefix_length, max_prefix_length)
len(dt_train_prefixes), len(dt_test_prefixes)

(731394, 200827)

## Task 3
Train an **XGboost** and a **Random Forest** models using *single bucketing* and *combined encoding*.

- Task 3: Train including column WIP.

### Bucketer: single

In [16]:
random_state = 123
bucket_method = 'single' # prefix, state, single, cluster

Single bucketing means no bucketing, so `bucket_data = dt_train_prefixes`:

In [17]:
def get_label_numeric(data):
    y = data.groupby(log_schema.get('case_id_col')).first()[log_schema.get('label')] # one row per case
    return [1 if label == log_schema.get('negative_outcome') else 0 for label in y]

bucket_data = dt_train_prefixes
train_y = get_label_numeric(bucket_data)
bucket_data_test = dt_test_prefixes
test_y = get_label_numeric(bucket_data_test)

### Encoder: combined

In [18]:
encoding_dict = {
    "laststate": ["static", "last"],
    "agg": ["static", "agg"],
    "index": ["static", "index"],
    "combined": ["static", "last", "agg"]
}

methods = encoding_dict['combined']

In [19]:
# Task 3: include column 'wip'
log_schema['dynamic_num_cols'].append('wip')
feature_combiner = FeatureUnion(
    [(method, EncoderFactory.get_encoder(method, **log_schema)) for method in methods])

### Models: Random Forest & XGBClassifier

In [20]:
RF = RandomForestClassifier(n_estimators=500, max_features=0.5, random_state=random_state)
XGB = XGBClassifier(
    n_estimators=400, learning_rate=0.3, n_jobs=2, use_label_encoder=False, 
    eval_metric='logloss', random_state=random_state)

### Pipelines & training

In [21]:
pipelines = {
    "RF": Pipeline([('encoder', feature_combiner), ('cls', RF)]),
    "XGB": Pipeline([('encoder', feature_combiner), ('cls', XGB)])
}

for model in pipelines:
    pipe = pipelines[model]
    pipe.fit(bucket_data, train_y)
    preds_pos_label_idx = np.where(pipe.classes_ == 1)[0][0]
    preds = pipe.predict_proba(bucket_data_test)[:,preds_pos_label_idx]
    score = roc_auc_score(test_y, preds)
    print(model, ": ", score)
    

RF :  0.8713168019560082
XGB :  0.8827917021279907


## Task 4
Train an **XGboost** and a **Random Forest** models using *single bucketing* and *combined encoding*.

- Task 4: Train excluding column WIP.

### Encoder: combined

In [22]:
# Task 4: exclude column 'wip'
log_schema['dynamic_num_cols'].remove('wip')
feature_combiner = FeatureUnion(
    [(method, EncoderFactory.get_encoder(method, **log_schema)) for method in methods])

### Models: Random Forest & XGBClassifier

In [23]:
RF = RandomForestClassifier(n_estimators=500, max_features=0.5, random_state=random_state)
XGB = XGBClassifier(
    n_estimators=400, learning_rate=0.3, n_jobs=2, use_label_encoder=False, 
    eval_metric='logloss', random_state=random_state)

### Pipelines & training

In [24]:
pipelines = {
    "RF": Pipeline([('encoder', feature_combiner), ('cls', RF)]),
    "XGB": Pipeline([('encoder', feature_combiner), ('cls', XGB)])
}

for model in pipelines:
    pipe = pipelines[model]
    pipe.fit(bucket_data, train_y)
    preds_pos_label_idx = np.where(pipe.classes_ == 1)[0][0]
    preds = pipe.predict_proba(bucket_data_test)[:,preds_pos_label_idx]
    score = roc_auc_score(test_y, preds)
    print(model, ": ", score)
    

RF :  0.8838849976711157
XGB :  0.885000641450972


**Compare the resulting models with the previous step and explain the differences in the results. Did the use of the WIP attribute affect the accuracy of the models? If so, explain why this effect can occur.**

The results are almost same for both situations, including the 'wip' column and excluding the 'wip' column. Results are slightly better when excluding the 'wip'. The 'wip' represents currently active nbr of cases. I think this is quite external value for activity as it don't describe the particular activity but more the whole situation. The 'wip' attribute confuses the prediction of particular case in particular activity.